In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

# Custom functions
from src.data_split import create_masks_data_split_per_months
from src.paths import PREPROCESSED_DATA_DIR

import numpy as np
import pandas as pd

In [3]:
df = pd.read_parquet(PREPROCESSED_DATA_DIR / 'modeling_dataset.parquet')

In [4]:
def fbeta_score_under_constraint(recall: float, precision: float, beta: float) -> float:
    return (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)


def ave_savings_score(y_true, score, amount, cf=5):
    #TODO: update description
    """Savings score.
   
    Parameters
    ----------
    y_true : array-like or label indicator matrix
        Ground truth (correct) labels.
    score : array-like or label indicator matrix
        Probabilities returned by a classifier.
    amount : array-like of shape = [n_samples, 4]
        Cost matrix of the classification problem
        Where the columns represents the costs of: false positives, false negatives,
        true positives and true negatives, for each example.
    cf    : float, default=10
        Fixed cost of investigation.
    Returns
    -------
    expected savings : float
        Savings of a using y_pred on y_true with cost-matrix cost-mat
        The best performance is 1.

    """

    ave_savings = np.sum(y_true*score*amount - score*cf)/np.sum(y_true*amount)
  
    return ave_savings


def savings_score(y_true, amount, cf=5):
    #TODO: update description
    """Savings score.
   
    Parameters
    ----------
    y_true : array-like or label indicator matrix
        Ground truth (correct) labels.
    amount : array-like of shape = [n_samples, 4]
        Cost matrix of the classification problem
        Where the columns represents the costs of: false positives, false negatives,
        true positives and true negatives, for each example.
    cf    : float, default=10
        Fixed cost of investigation.
    Returns
    -------
    expected savings : float
        Savings of a using y_pred on y_true with cost-matrix cost-mat
        The best performance is 1.

    """

    savings = np.sum(y_true*amount - len(y_true)*cf)/np.sum(y_true*amount)
  
    return savings


In [5]:
train_masks, val_masks, test_masks = create_masks_data_split_per_months(df,
                                                                        n_splits=3,
                                                                        validation_ratio=0.33,
                                                                        offset_trainval=5,
                                                                        offset_test=1)

In [6]:
cat_vars = ['errors','use_chip','foreign_transaction', 'morning', 'afternoon','day_week']
high_cat_vars = ['mcc']
num_vars = ['amount','amount_log','recency','frequency','monetary']

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import TargetEncoder
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

import xgboost as xgb


class TopCategoriesEncoder(BaseEstimator, TransformerMixin):
  def __init__(self, n=5):
    self.n = n
    self.top_categories = []

  def fit(self, X, y=None):
    # Calculate the proportion of the positive class for each category
    
    proportions = self.group_mean(X, y)

    # Sort the categories by the proportion of the positive class
    sorted_categories = dict(sorted(proportions.items(), key=lambda item: item[1], reverse=True))

    # Store the top n categories
    self.top_categories = list(sorted_categories.keys())[:self.n]

    return self
    
  def transform(self, X):
    # For each top category, add a new binary feature to X
    X = X.flatten()
    X_trans = np.zeros((X.shape[0], len(self.top_categories)))
    for i, category in enumerate(self.top_categories):
    
      X_trans[:, i] = (X == category)

    return X_trans

  @staticmethod
  def group_mean(X, y):
    # Get unique categories
    categories = np.unique(X)
    X = X.flatten()

    # Compute mean for each category
    means = {category: np.mean(y[X == category]) for category in categories}
  
    return means
    
class AddIsolationForestAnomalyScore(BaseEstimator, TransformerMixin):
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)
    
  def fit(self, X, y=None):
    n_estimators = getattr(self, 'n_estimators', 100)
    
    self.iforest = IsolationForest(n_estimators=n_estimators)
    self.iforest.fit(X)
    return self

  def transform(self, X):
    anomaly_scores = self.iforest.decision_function(X).reshape(-1, 1)
    return np.hstack([X, anomaly_scores])
      
      
class CatNarrow(BaseEstimator, TransformerMixin):
  "Narrows the categories of a categorical variable based on a threshold."

  def __init__(self, threshold: float=0.05):
    self.threshold = threshold
    
  def fit(self, X, y=None):
    X_ = np.array(X).astype(str)
    self.dc_filtered  = []
    
    # For each column, check the frequency of each category and filter based on threshold
    for c in range(X_.shape[1]):
      X_aux = X_[:, c]
      unique, counts = np.unique(X_aux, return_counts=True)
      counts_normalized = counts / sum(counts)
      dc = dict(zip(unique, counts_normalized))
      dc_filtered_ = {k: v for k, v in dc.items() if v >= self.threshold}  # Only keep categories with frequency >= threshold
      dc_filtered_ = np.array(list(dc_filtered_.keys())).reshape(-1,1)
      self.dc_filtered.append(dc_filtered_)
    
    return self
  
  def transform(self, X):
    """Replace categories with frequency < threshold by 'undefined'.
    
    Args:
      X (np.ndarray): The input array containing categories.
      
    Returns:
      X_trans (np.ndarray): The transformed array with replaced categories.
    """
       
    X_ = np.array(X).astype(str)
    list_aux = []
    
    for c in range(X_.shape[1]):  
      X_aux = np.array(X_[:, c]).reshape(-1,1)
      dc_filtered_ = self.dc_filtered[c]
      np.place(X_aux, np.isin(X_aux, dc_filtered_, invert = True), ['undefined']) # Replace categories with frequency < threshold by 'undefined'
      list_aux.append(X_aux)
    
    X_trans = np.concatenate(list_aux, axis = 1)
    return X_trans
  


num_transformer = Pipeline(steps = [
                                    ('imputer', SimpleImputer(strategy='median')),
                                    ('scaler', StandardScaler())
                                    ]
                          )

cat_transformer = Pipeline(steps = [
                                    ('imputer', SimpleImputer(
                                                              strategy='constant',
                                                              fill_value=-9999
                                                              )
                                    ),
                                    ('catnarrow', CatNarrow(threshold = 0.05)),
                                    ('target_enc', TargetEncoder()),
                                    ('scaler', StandardScaler())
                                            ]
                                   )

high_cat_transformer = Pipeline(steps = [
                                    ('imputer', SimpleImputer(
                                                              strategy='constant',
                                                              fill_value=-9999
                                                              )
                                    ),
                                    ('topcat_enc', TopCategoriesEncoder(n=3)),
                                    ('target_enc', TargetEncoder()),
                                    ('scaler', StandardScaler())
                                            ]
                                   )
                                   

processing = ColumnTransformer(transformers=[
                                            ('cat', cat_transformer, cat_vars),
                                            ('high_cat', high_cat_transformer, high_cat_vars),
                                            ('num', num_transformer, num_vars)
                                            ],
                               remainder='drop'
                              )
pipe = Pipeline(steps=[
                      ('processing', processing),
                      ('isolation_forest', AddIsolationForestAnomalyScore(n_estimators=100,
                                                                          random_state=123))
                       ]
              )

In [8]:
pipe 

Pipeline(steps=[('processing',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=-9999,
                                                                                 strategy='constant')),
                                                                  ('catnarrow',
                                                                   CatNarrow()),
                                                                  ('target_enc',
                                                                   TargetEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['errors', 'use_chip',
                                                   'foreign_transaction',
                                                   'morning', 'afternoon',
                                                   'day_week']),
                                                 ('high_cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=-9999,
                                                                                 strategy='constant')),
                                                                  ('topcat_enc',
                                                                   TopCategoriesEncoder(n=3)),
                                                                  ('target_enc',
                                                                   TargetEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['mcc']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['amount', 'amount_log',
                                                   'recency', 'frequency',
                                                   'monetary'])])),
                ('isolation_forest', AddIsolationForestAnomalyScore())])

## Optimizing PR AUC

In [9]:
import optuna
from sklearn.metrics import average_precision_score, precision_score, recall_score, roc_auc_score, f1_score, fbeta_score

list_fbeta_score = []
list_precision = []
list_recall = []
list_best_fbeta_score = []

train_masks, val_masks, test_masks = create_masks_data_split_per_months(df,
                                                                    n_splits=3,
                                                                    offset_trainval=5,
                                                                    offset_test=1
                                                                    )
def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "n_estimators": trial.suggest_int("n_estimators", 2, 50),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "learning_rate": trial.suggest_float("learning_rate", 0.10, 1.0),
    }

    scores = []
    splits = train_masks.shape[0]
    for i in range(splits):
        
        # split data for training and validation
        X_train = df.drop(columns=['fraud']).loc[train_masks[i]]
        y_train = df['fraud'].loc[train_masks[i]]
        
        X_val = df.drop(columns=['fraud']).loc[val_masks[i]]
        y_val = np.asarray(df['fraud'].loc[val_masks[i]]).astype(int)

        pipeline_training = pipe.fit(X_train, y_train)
        
        X_train = pipeline_training.transform(X_train)

        X_val = pipeline_training.transform(X_val)

        
        # train the model
        m0_xgb = xgb.XGBClassifier(**hyperparams,
                                    random_state=123, 
                                    n_jobs=-1
                                    )
        
        
        m0_xgb.fit(X_train, y_train, verbose=False)
        
        
        # evaluate the model
        y_proba = m0_xgb.predict_proba(X_val)[:, 1]
        ap = average_precision_score(y_val, y_proba)
        scores.append(ap)
   
    # Return the mean score
    return np.array(scores).mean()

In [10]:
study = optuna.create_study(direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=123)
                            )
study.optimize(objective, n_trials=5)

[I 2024-03-25 16:25:05,766] A new study created in memory with name: no-name-cfa745c4-9fa6-491b-9348-104ec1a40c46
[I 2024-03-25 16:25:10,513] Trial 0 finished with value: 0.5595792440092217 and parameters: {'n_estimators': 36, 'max_depth': 3, 'learning_rate': 0.3041663082077828}. Best is trial 0 with value: 0.5595792440092217.
[I 2024-03-25 16:25:15,738] Trial 1 finished with value: 0.5835899146252183 and parameters: {'n_estimators': 29, 'max_depth': 4, 'learning_rate': 0.4807958141120149}. Best is trial 1 with value: 0.5835899146252183.
[I 2024-03-25 16:25:20,593] Trial 2 finished with value: 0.5992884084325633 and parameters: {'n_estimators': 50, 'max_depth': 4, 'learning_rate': 0.5328387113359249}. Best is trial 2 with value: 0.5992884084325633.
[I 2024-03-25 16:25:25,369] Trial 3 finished with value: 0.5111264837460139 and parameters: {'n_estimators': 21, 'max_depth': 3, 'learning_rate': 0.7561447366456374}. Best is trial 2 with value: 0.5992884084325633.
[I 2024-03-25 16:25:30,203

In [11]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'n_estimators': 50, 'max_depth': 4, 'learning_rate': 0.5328387113359249}


# Evaluation

In [12]:
split = 2


trainval_masks = train_masks[split] + val_masks[split]
X_trainval = df.drop(columns=['fraud']).loc[trainval_masks]
y_trainval = df['fraud'].loc[trainval_masks]
X_train = df.drop(columns=['fraud']).loc[train_masks[split]]
y_train = df['fraud'].loc[train_masks[split]]
X_val = df.drop(columns=['fraud']).loc[val_masks[split]]
y_val = np.asarray(df['fraud'].loc[val_masks[split]]).astype(int)
X_test = df.drop(columns=['fraud']).loc[test_masks[split]]
y_test = np.asarray(df['fraud'].loc[test_masks[split]]).astype(int)

amount_test = df['amount'].loc[test_masks[split]].values.reshape(-1,)
amount_val = df['amount'].loc[val_masks[split]].values.reshape(-1,)


In [13]:
X_trainval = pipe.fit(X_trainval, y_trainval).transform(X_trainval)

X_train = pipe.fit(X_train, y_train).transform(X_train)
X_val = pipe.transform(X_val)
X_test = pipe.transform(X_test)

In [14]:
m0_trainval = xgb.XGBClassifier(**best_params,
                            random_state=123, 
                            n_jobs=-1
                            )

m0_trainval.fit(X_trainval, y_trainval, verbose=False)
pred_test_m0 = m0_trainval.predict_proba(X_test)[:, 1]

In [15]:
ix_sorted_top = np.argsort(pred_test_m0)[::-1]
topk = 0.01
y_test_ix = y_test[ix_sorted_top][:int(topk*len(y_test))]
amount_test_ix = amount_test[ix_sorted_top][:int(topk*len(y_test))]

print('ROC-AUC: ', roc_auc_score(y_test, pred_test_m0))
print('PR-AUC: ', average_precision_score(y_test, pred_test_m0))
print('Expected Savings: ', ave_savings_score(y_test, pred_test_m0, amount_test))
print('Uplift: ', average_precision_score(y_test, pred_test_m0)/np.mean(y_test))
# print('F1-score: ', f1_score(y_test, c > np.mean(y_train==0)))
print('Recall Top-1%: ', 100*np.sum(y_test_ix)/np.sum(y_test))
print('Precision Top-1%', 100*np.mean(y_test_ix))
print('Recall Monetary Top-1%: ',100*np.sum(y_test_ix*amount_test_ix)/np.sum(y_test*amount_test))

ROC-AUC:  0.9967642710832739
PR-AUC:  0.23268396952655448
Expected Savings:  0.012364053526646736
Uplift:  216.88966371626717
Recall Top-1%:  93.93939393939394
Precision Top-1% 10.097719869706841
Recall Monetary Top-1%:  99.31780560379408


# Optimal Threshold

In [16]:
m0_train = xgb.XGBClassifier(**best_params,
                            random_state=123, 
                            n_jobs=-1
                            )

m0_train.fit(X_train, y_train, verbose=False)
pred_val_m0 = m0_train.predict_proba(X_val)[:, 1]

In [17]:
list_f1_score = []
list_recall = []
list_precision = []
for i in np.linspace(0, 1, 100):
    threshold = i
    pred_labels = (pred_val_m0 > threshold).astype(int)
    list_f1_score.append(f1_score(y_val, pred_labels)) 
    list_recall.append(recall_score(y_val, pred_labels))
    list_precision.append(precision_score(y_val, pred_labels))

my_dict = dict(zip(np.linspace(0, 1, 100), list_f1_score))

# Get the key with the maximum value
max_key = max(my_dict, key=my_dict.get)
index_max_key = list(my_dict.keys()).index(max_key)

optimal_threshold = max_key
print(f'optimal_threshold={optimal_threshold}')  

print('Max F1 score on test set: ', max(my_dict.values()))
print('Recall on test set: ', list_recall[index_max_key])
print('Precision on test set: ', list_precision[index_max_key])
print('Number of predicted positives proportional number of obs: ', np.sum((pred_val_m0 > optimal_threshold).astype(int))/y_val.shape[0])

optimal_threshold=0.6666666666666667
Max F1 score on test set:  0.4489795918367347
Recall on test set:  0.4074074074074074
Precision on test set:  0.5
Number of predicted positives proportional number of obs:  0.0007388252678241596


# Optimal Thresholding under Constraint

In [18]:
def fbeta_score_under_constraint(recall: float, precision: float, beta: float) -> float:
    return (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
beta_ = 2

In [25]:
number_split = val_masks.shape[0]
topk = 0.001

list_fbeta_score = []
list_fbeta_max_score = []
list_precision = []
list_recall = []
list_recall_money = []
list_optimal_threshold = []
for split in range(number_split):


    X_train = df.drop(columns=['fraud']).loc[train_masks[split]]
    y_train = df['fraud'].loc[train_masks[split]]
    X_val = df.drop(columns=['fraud']).loc[val_masks[split]]
    y_val = np.asarray(df['fraud'].loc[val_masks[split]]).astype(int)
    
    X_train = pipe.fit(X_train, y_train).transform(X_train)
    X_val = pipe.transform(X_val)

    amount_val = df['amount'].loc[val_masks[split]].values.reshape(-1,)
    
    m0_train = xgb.XGBClassifier(**best_params,
                            random_state=123, 
                            n_jobs=-1
                            )

    m0_train.fit(X_train, y_train, verbose=False)
    pred_val_m0 = m0_train.predict_proba(X_val)[:, 1]
    
    n_top1perc = np.round((len(pred_val_m0[np.argsort(pred_val_m0)[::-1]])*topk),0).astype(int)
    ix_sorted_top_val = np.argsort(pred_val_m0)[::-1]
    
    for i in range(1,n_top1perc+1):

        pseudo_labels = np.ones_like(pred_val_m0[ix_sorted_top_val[:i]]).astype(int)
        
        precision_under_constraint = precision_score(y_val[ix_sorted_top_val[:i]], pseudo_labels) 
        list_precision.append(precision_under_constraint)
        
        recall_under_constraint = np.sum(y_val[ix_sorted_top_val[:i]])/np.sum(y_val) 
        list_recall.append(recall_under_constraint)
        
        recall_monetary_under_constraint = np.sum(y_val[ix_sorted_top_val[:i]]* \
                                                amount_val[ix_sorted_top_val[:i]]) / \
                                                np.sum(y_val*amount_val)
        list_recall_money.append(recall_monetary_under_constraint)
        
        fbeta_score_ = fbeta_score_under_constraint(recall_under_constraint, precision_under_constraint, beta_)
        list_fbeta_score.append(fbeta_score_)
    
    my_dict = dict(zip(np.sort(pred_val_m0)[::-1], list_fbeta_score))
    max_key = max(my_dict, key=my_dict.get)
    optimal_threshold = max_key
    
    list_optimal_threshold.append(optimal_threshold)
    list_fbeta_max_score.append(max(my_dict.values()))

optimal_treshold_test = np.median(list_optimal_threshold)
print(f'optimal treshold on test={optimal_treshold_test}')

optimal treshold on test=0.4558947682380676


In [31]:
number_split = test_masks.shape[0]

list_recall_test_ = []
list_precision_test_ = []
list_proportion_investigations_ = []

for split in range(number_split):

    trainval_masks = train_masks[split] + val_masks[split]

    X_trainval = df.drop(columns=['fraud']).loc[trainval_masks]
    y_trainval = df['fraud'].loc[trainval_masks]
    
    X_test = df.drop(columns=['fraud']).loc[test_masks[split]]
    y_test = np.asarray(df['fraud'].loc[test_masks[split]]).astype(int)

    X_trainval = pipe.fit(X_trainval, y_trainval).transform(X_trainval)
    X_test = pipe.transform(X_test)

    amount_test = df['amount'].loc[test_masks[split]].values.reshape(-1,)
    
    m0_trainval = xgb.XGBClassifier(**best_params,
                                    random_state=123, 
                                    n_jobs=-1
                                    )

    m0_trainval.fit(X_trainval, y_trainval, verbose=False)
    pred_test_m0 = m0_trainval.predict_proba(X_test)[:, 1]
    
    label_pred_test = (pred_test_m0 > optimal_treshold_test).astype(int)
    n_top1perc = np.round((len(pred_test_m0[np.argsort(pred_test_m0)[::-1]])*topk),0).astype(int)
    recall_monetary_test_set = np.sum(label_pred_test*amount_test*y_test)/np.sum(y_test*amount_test)

    recall_score_ = recall_score(y_test, label_pred_test)
    precision_score_ = precision_score(y_test, label_pred_test)
    proportion_investigations = np.sum((pred_test_m0 > optimal_threshold).astype(int))/y_test.shape[0]

    list_recall_test_.append(recall_score_)
    list_precision_test_.append(precision_score_)
    list_proportion_investigations_.append(proportion_investigations)
    
mean_recall = np.mean(list_recall_test_)
mean_precision = np.mean(list_precision_test_)
mean_recall_money = np.mean(recall_monetary_test_set)
mean_prop_invs = np.mean(list_proportion_investigations_)

print(f'Recall on test set: {100*mean_recall:.2f}%')
print(f'Precision on test set: {100*mean_precision:.2f}%')
print(f'Recall Monetary on test set: {100*mean_recall_money:.2f}%')
print(f'Proportion {100*mean_prop_invs:.2f}% out of the whole test set, and out of {100*(mean_prop_invs/topk):.2f}% investigations permitted')

Recall on test set: 36.30%
Precision on test set: 39.47%
Recall Monetary on test set: 40.48%
Proportion 0.06% out of the whole test set, and out of 64.72% investigations permitted
